In [50]:
from tensorflow.keras.layers import Input, LSTM, GRU, SimpleRNN, Dense, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier
from sklearn.model_selection import RepeatedStratifiedKFold

In [51]:
files_path = 'CleanFiles/'
files_format = '.csv'
files_postfix = 'Extract'
file_names = ['AgiaParaskevi','Aristotelous','Athens','Elefsina','Koropi','Liosia',
              'Lykovrisi','Marousi','NeaSmirni','Patision','Peristeri',
              'Pireus','Thrakomakedones']

In [52]:
def pm10_index(val):
    if val <= 25.0:
        return 0
    elif 26.0 <= val <= 50.0:
        return 1
    elif 51.0 <= val <= 90.0:
        return 2
    elif 91.0 <= val <= 180.0:
        return 3
    else:
        return 4

In [53]:
def get_time(tmp):
    return tmp[11:13]

In [54]:
#neasmirni = pd.read_csv('CleanFiles/NeaSmirniExtract.csv', sep=',')
#neasmirni = neasmirni[['date_time','station_id','season','real_temp','real_windspd','real_winddir','real_humidity','forecast_tempC','forecast_windSpeed','forecast_windDirection','forecast_humidity','pm10']]
#neasmirni['pm10'] = neasmirni['pm10'].map(lambda a: pm10_index(a))
#temp = pd.read_csv('CleanFiles/PeristeriExtract.csv', sep=',')

In [62]:
T = 72
D = 1
X = []
Y = []
for t in range(len(neasmirni) - T):
    x = neasmirni[t:t+T]
    X.append(x)
    y = neasmirni[t:t+T]
    Y.append(y)

In [63]:
X

[    Unnamed: 0 date_time  real_temp  real_humidity  real_windspd  station_id  \
 0            0       NaN        0.0            3.0           2.0           3   
 1            1       NaN        0.0            4.0           3.0           3   
 2            2       NaN       -1.0            4.0           1.0           3   
 3            3       NaN        0.0            3.0           2.0           3   
 4            4       NaN       -1.0            4.0           2.0           3   
 ..         ...       ...        ...            ...           ...         ...   
 67          67       NaN        1.0            4.0           2.0           3   
 68          68       NaN        1.0            4.0           2.0           3   
 69          69       NaN        2.0            4.0           1.0           3   
 70          70       NaN        1.0            4.0           2.0           3   
 71          71       NaN        1.0            4.0           2.0           3   
 
     forecast_tempC  forec

In [60]:
def loadToModel(files, formating, pathFiles, postfx):
    T = 72
    D = 1
    X = []
    Y = []
    for fileName in files:
        accuracy = 0
        df = pd.read_csv(pathFiles + fileName + postfx + formating, sep=',')
        df['pm10'] = df['pm10'].map(lambda a: pm10_index(a))
        #df['date_time'] = df['date_time'][]
        for t in range(len(neasmirni) - T):
            X.append(df[t:t+T][['date_time','station_id','season','real_temp','real_humidity','real_windspd','real_winddir','forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection']])
            Y.append(df[t:t+T][['pm10']])
        model = LGBMClassifier()
        cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
        n_scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1)
        print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [61]:
loadToModel(file_names, files_format, files_path, files_postfix)

ValueError: cannot copy sequence with size 72 to array axis with dimension 1